In [1]:
import os

import numpy as np

import tensorflow as tf
from tensorflow import keras



from RoadDataset import RoadDataset

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
#val_datasets_path = "../notebooks/datasets/validation/"
val_datasets_path = "../notebooks/datasets/train/"
sub_dirs = os.listdir(val_datasets_path)
val_dataset_paths = [[val_datasets_path + sub_dir + "/" + sub_dir + ".csv", val_datasets_path + sub_dir + "/frames/"] for sub_dir in sub_dirs]
val_dataset_paths

[['../notebooks/datasets/train/Sun Dec 27 18_09_33 2020/Sun Dec 27 18_09_33 2020.csv',
  '../notebooks/datasets/train/Sun Dec 27 18_09_33 2020/frames/'],
 ['../notebooks/datasets/train/Sun Dec 27 17_46_56 2020/Sun Dec 27 17_46_56 2020.csv',
  '../notebooks/datasets/train/Sun Dec 27 17_46_56 2020/frames/'],
 ['../notebooks/datasets/train/Sun Feb 21 21:50:50 2021/Sun Feb 21 21:50:50 2021.csv',
  '../notebooks/datasets/train/Sun Feb 21 21:50:50 2021/frames/'],
 ['../notebooks/datasets/train/Sun Dec 27 18_01_53 2020/Sun Dec 27 18_01_53 2020.csv',
  '../notebooks/datasets/train/Sun Dec 27 18_01_53 2020/frames/'],
 ['../notebooks/datasets/train/Wed Dec 30 17_34_37 2020/Wed Dec 30 17_34_37 2020.csv',
  '../notebooks/datasets/train/Wed Dec 30 17_34_37 2020/frames/'],
 ['../notebooks/datasets/train/Sun Jan 10 11:31:18 2021/Sun Jan 10 11:31:18 2021.csv',
  '../notebooks/datasets/train/Sun Jan 10 11:31:18 2021/frames/'],
 ['../notebooks/datasets/train/Sun Dec 27 18_25_24 2020/Sun Dec 27 18_25_24 

In [4]:
dataset_index = 28
column_names = ['filename','steering_wheel_angle','speed', 'blink_l', 'blink_r']
val_set = RoadDataset(val_dataset_paths[dataset_index][0], val_dataset_paths[dataset_index][1], column_names, batch_size=16)
val_set.clean()

Reading csv file: ../notebooks/datasets/train/Sun Jan 10 11:30:59 2021/Sun Jan 10 11:30:59 2021.csv
Loaded dataset with 31859 samples
Cleaning dataset. Starting with 31859 samples.
Removing 0 rows for reason: SWA is NaN
Removing 3 rows for reason: No frame after cam latency compensation
Removing 689 rows for reason: High SWA (> +180/ < -180 deg)
Removing 2788 rows for reason: Low speed (< 25 km/h)
Removing 312 rows for reason: Left blinker on
Removing 1066 rows for reason: Right blinker on
27001 samples remaining.


In [5]:
stats = val_set.get_csv().describe()
stats = stats.transpose()
stats

,count,mean,std,min,25%,50%,75%,max
steering_wheel_angle,27001.0,-1.395908,13.047496,-112.880,-5.070,-1.000,2.630,120.650
speed,27001.0,69.584447,26.628558,25.048,50.344,65.348,77.469,150.474
blink_l,27001.0,0.000000,0.000000,0.000,0.000,0.000,0.000,0.000
blink_r,27001.0,0.000000,0.000000,0.000,0.000,0.000,0.000,0.000


In [6]:
#val_set.balance()
#val_set.normalize()

In [7]:
stats = val_set.get_csv().describe()
stats = stats.transpose()
stats

,count,mean,std,min,25%,50%,75%,max
steering_wheel_angle,27001.0,-1.395908,13.047496,-112.880,-5.070,-1.000,2.630,120.650
speed,27001.0,69.584447,26.628558,25.048,50.344,65.348,77.469,150.474
blink_l,27001.0,0.000000,0.000000,0.000,0.000,0.000,0.000,0.000
blink_r,27001.0,0.000000,0.000000,0.000,0.000,0.000,0.000,0.000


In [8]:
#model_name = 'diypilot_v1'
#model_name = 'diypilot_v2_epoch_10'
#model_name = 'diypilot_v3_epoch_5'
#model_name = 'openpilot_replica_epoch_59'
#model_name = 'openpilot_replica_mae_epoch_100'
#model_name = 'diypilot_v9_small_FC_epoch_3'
#model_name = 'diypilot_v11_random_shifts_epoch_3'
#model_name = 'diypilot_v11_random_shifts_full_balance_epoch_3'
#model_name = 'diypilot_v11_full_balance_epoch_163'
#model_name = 'diypilot_v12_epoch_58'
#model_name = 'diypilot_v13_sum_min5_overrepr_epoch_10'
#model_name = 'diypilot_v14_smaller_crop_epoch_3'
#model_name = 'diypilot_v14_smaller_crop_wider_epoch_3'
#model_name = 'diypilot_v14_smaller_crop_wider_white_corners_epoch_10'
#model_name = 'diypilot_v14_smaller_crop_wider_white_corners_random_shift_epoch_5'
model_name = 'v17.6_symmetric_balancing_no_flip_epoch_18'
loaded_model = keras.models.load_model('./trained_models/' + model_name + '.h5')
loaded_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
frame_in (InputLayer)           [(None, 170, 640, 3) 0                                            
__________________________________________________________________________________________________
frame_normalize (Lambda)        (None, 170, 640, 3)  0           frame_in[0][0]                   
__________________________________________________________________________________________________
frame_downsample (MaxPooling2D) (None, 85, 213, 3)   0           frame_normalize[0][0]            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 27, 70, 24)   1824        frame_downsample[0][0]           
____________________________________________________________________________________________

In [9]:
import sys
sys.path.append("..") # Adds higher directory to python modules path.
from Jetson.autopilot.record.AutopilotGUI import AutopilotGUI

In [10]:
gui = AutopilotGUI()

In [11]:
def show_frame(test_image_index):

    test_sample = val_set.get_frame(test_image_index)
    test_image = test_sample[0]
    test_image_cropped = val_set.get_cropped_frame(test_image_index, shift=0)[0]
    test_labels = test_sample[1]

    print("Test csv: " + str(test_labels))

    test_data = np.expand_dims(test_image_cropped, axis=0)
    predictions = loaded_model.predict(test_data)

    gui.set_frame(test_image)
    gui.set_show_overlay(True)
    gui.set_engaged(False)
    gui.set_recording(True)

    timestamp = test_labels['filename']
    gui.set_timestring(timestamp)

    velocity = test_labels['speed']
    gui.set_velocity(velocity)

    actual_swa = test_labels['steering_wheel_angle']*90
    gui.set_actual_swa(actual_swa)

    predicted_swa = predictions[0][0]*90
    
    swa_error = predicted_swa - actual_swa
    next_shift = swa_error / val_set.get_shift_multiplier()
    
    next_frame = val_set.get_cropped_frame(test_image_index+1, shift=next_shift)[0]
    
    
    print("Ground-truth: " +str(actual_swa))
    print("Prediction: " + str(predicted_swa))
    gui.set_predicted_swa(predicted_swa)

    gui.show_jupyter()

In [12]:
import ipywidgets as widgets
frame_slider = widgets.IntSlider(min=0, max=val_set.get_batch_size()*len(val_set)-1, continuous_update=False)

In [13]:
from ipywidgets import interact

interact(show_frame, test_image_index=frame_slider);

interactive(children=(IntSlider(value=0, continuous_update=False, description='test_image_index', max=26991), …

In [14]:
def sim_step(frame_idx, last_shift):
    test_sample = val_set.get_frame(frame_idx)
    test_image = test_sample[0]
    test_image_cropped = val_set.get_cropped_frame(frame_idx, shift=last_shift, offset=0)[0]
    test_image_cropped_unshifted = val_set.get_cropped_frame(frame_idx, shift=0, offset=0)[0]
    test_labels = test_sample[1]

    test_data = np.expand_dims(test_image_cropped, axis=0)
    predictions = loaded_model.predict(test_data)
    
    timestamp = test_labels['filename']
    gui.set_timestring(timestamp)

    actual_swa = test_labels['steering_wheel_angle']*90
    predicted_swa = predictions[0][0]*90
    
    swa_error = predicted_swa - actual_swa
    this_shift = -swa_error #/ val_set.get_shift_multiplier()
    
    next_frame = val_set.get_cropped_frame(frame_idx+1, shift=this_shift + last_shift, offset=0)[0]
    
    #gui.set_frame(next_frame)
    #gui.set_show_overlay(True)
    #gui.show_jupyter()
    
    return this_shift, next_frame, test_image_cropped_unshifted

In [15]:
import cv2
import time
fourcc = cv2.VideoWriter_fourcc(*'XVID')
filename = model_name + "__" + str(time.time())
print(filename)
out = cv2.VideoWriter(filename + '.avi',fourcc, 15.0, (848,340))

total_shift = 0
aborts = 0
for frame_idx in range(5000):
    shift, frame, unshifted_frame = sim_step(frame_idx, total_shift)
    total_shift += shift
    
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    video_frame = np.vstack((unshifted_frame, frame))
    out.write(video_frame.astype(np.uint8))
    
    print(frame_idx, total_shift)
    
    if (abs(total_shift) > 150):
        
        # abort, insert 1s black in video to visualize failure and restart with 0 shift
        dummy_frame = np.zeros((340,848,3), np.uint8)
        
        for i in range(15):
            out.write(dummy_frame)
            
        total_shift = 0
        aborts += 1
    
out.release()

print("Aborts: " + str(aborts))

v17.6_symmetric_balancing_no_flip_epoch_18__1620977886.4216192


AttributeError: 'RoadDataset' object has no attribute 'get_frame'